# Imports
---
Import new skis / snowboards to an SQL database

In [32]:
import json
import pymysql
import re

In [33]:
# Database Setup
# --------------------------------------------------
def setupdb():
    f = open('../app/config/localdb_config.json')
    dbconfig = json.loads(f.read())
    db = pymysql.connect(host=dbconfig['localhost'], user=dbconfig['username'], password=dbconfig['password'], database=dbconfig['database'])
    f.close()
    return db

In [34]:
db = setupdb()
cursor = db.cursor()

In [35]:
params = {
    'size': ['size', 'length'],
    'nose width': ['nose width', 'tip width'],
    'waist width': ['waist width'],
    'tail width': ['tail width'],
    'sidecut': ['sidecut radius', 'turning radius', 'radius', 'sidecut'],
    'setback': ['stance setback', 'setback'],
    'effective edge': ['effective edge', 'running length']
}

In [36]:
values = {
    'size': '',
    'nose width': '',
    'waist width': '',
    'tail width': '',
    'sidecut': '',
    'setback': '',
    'effective edge': ''
}

In [83]:
# Extraction of data 
# from string format to a tabular structure
# --------------------------------------------------
def extract_data(table):
    # Remove units from headings
    while ')' in table:
        table = table[:table.find('(')] + table[table.find(')') + 1:]
        
    breakpoints = []
    
    # Find location of headings in table
    for i in params:
        # Check each param alias
        for j in params[i]:
            if j in table:
                breakpoints.append(table.find(j))
                break
        
        values[i] = []
        
    # Extract table data into rows
    rows = []
    breakpoints.sort()
    for i, b in enumerate(breakpoints):
        if i > 0:
            rows.append(table[breakpoints[i-1]:b])
            print(f"Row: {table[breakpoints[i-1]:b]}")
        if i == len(breakpoints) -1:
            rows.append(table[b:])
    
    
    # Remove lables from rows
    for row in rows:
        for i in params:
            for j in params[i]:
                if j in row:
                    values[i] = row.replace(j, '')
                    break
                    
    
    # Format table rows as lists
    for val in values:
        try:
            if val == 'sidecut':
                print(values[val])
            values[val] = values[val].strip()
            values[val] = values[val].replace('\u200b/\u200b', '/')
            values[val] = values[val].split() 
            
        except Exception as e:
            x = e
            
    
    # Truncate long rows to the correct size
    num_sizes = len(values['size'])
    for val in values:
        # Sidecuts can have more than one radius
        if val != "sidecut":
            values[val] = values[val][:num_sizes]

        # Remove rogue characters from row values
        if len(values[val]) > 0 and type(values[val][0]) == str:
            for x, v in enumerate(values[val]):
                values[val][x] = values[val][x].replace('\u200b', '')
        
        
        # Fill empty or missing data with null values
        for x in range(num_sizes - len(values[val])):
            
            values[val].append(0)
            
    
        
        
    return values

In [38]:
# Save skiboard in MySQL DB
# --------------------------------------------------
def save_skiboard(skiboard):
    try:
        sql = f"""INSERT INTO skiboards (
            url,
            brand,
            model,
            year,
            name,
            slug,
            category,
            family,
            description,
            stiffness,
            shape,
            flex_profile,
            camber_profile,
            camber_details,
            core,
            core_profiling,
            fibreglass,
            laminates,
            resin,
            base,
            edge_tech,
            topsheet,
            sidewall,
            inserts,
            asym,
            weight,
            womens,
            youth
        ) VALUES (
            '{skiboard['url']}',
            '{skiboard['brand']}',
            '{skiboard['model']}',
            '{skiboard['year']}',
            '{str(skiboard['brand']).title()} {str(skiboard['model']).title()} {str(skiboard['year'])}',
            '{str(skiboard['brand']).lower().replace(' ', '-')}-{str(skiboard['model']).lower().replace(' ', '-')}-{str(skiboard['year'])}',
            '{skiboard['category']}',
            '{skiboard['family']}',
            '{skiboard['description']}',
            '{skiboard['stiffness']}',
            '{skiboard['shape']}',
            '{skiboard['flex_profile']}',
            '{skiboard['camber_profile']}',
            '{skiboard['camber_details']}',
            '{skiboard['core']}',
            '{skiboard['core_profiling']}',
            '{skiboard['fibreglass']}',
            '{skiboard['laminates']}',
            '{skiboard['resin']}',
            '{skiboard['base']}',
            '{skiboard['edge_tech']}',
            '{skiboard['topsheet']}',
            '{skiboard['sidewall']}',
            '{skiboard['inserts']}',
            '{skiboard['asym']}',
            '{skiboard['weight']}',
            '{skiboard['womens']}',
            '{skiboard['youth']}'
        )"""
        cursor.execute(sql)
        db.commit()
    except Exception as e:
        print(f"ERROR:\n{e}")

In [39]:
# Save data in MySQL DB
# --------------------------------------------------
def save_values(skiboard, values):
    #print(f"Importing...\n{str(skiboard['brand']).capitalize()} {str(skiboard['model']).capitalize()} {str(skiboard['year'])}\nValues:\n{values}")

    for x in range(len(values['size'])):
        try:
            sql = f"""REPLACE INTO sizes (
                skiboard_id,
                size,
                nose_width,
                waist_width,
                tail_width,
                sidecut,
                setback,
                effective_edge
            ) VALUES (
                '{str(skiboard['id'])}',
                '{str(values['size'][x])}',
                '{float(values['nose width'][x])}',
                '{float(values['waist width'][x])}',
                '{float(values['tail width'][x])}',
                '{str(values['sidecut'][x])}',
                '{float(values['setback'][x])}',
                '{float(values['effective edge'][x])}'
            )"""

            cursor.execute(sql)
            db.commit()
        except Exception as e:
            print(f"ERROR: size ({values['size'][x]})\n{e}\n")
            print(f"SQL:\n{sql}\n\n")
    
    return True

# Create New SkiBoard
---

In [40]:
id_counter = 97

In [41]:
print(id_counter)

97


In [42]:
skiboard = {
    'id': id_counter,
    'url': '',
    'brand': 'Ride',
    'model': 'Manic',
    'year': 2020,
    'category': 'Snowboard',
    'family': '',
    'description': "Dance like you've never danced before with the Ride Manic Snowboard. This directional shredster is versatile, smooth, and super reliable when things get choppy. The lightweight aspen core is laid up with carbon stringers for added pop to fuel your moves, and the quadratic sidecut with generous tip rocker makes for easy turning through powder and chop. It's time to get wild!".replace("'", "`"),
    'stiffness': 3,
    'shape': 'Twin',
    'flex_profile': '',
    'camber_profile': 'Directional Hybrid Camber',
    'camber_details': '',
    'core': 'Foundation™ Core',
    'core_profiling': '',
    'fibreglass': 'Biaxial Glass',
    'laminates': 'Single Impact Plates ~ Linear Carbon',
    'base': 'Extruded, Stone Ground Base',
    'edge_tech': '',
    'topsheet': '',
    'resin': '',
    'sidewall': 'Slimewalls®',
    'inserts': '4x2',
    'asym': 0,
    'weight': 0,
    'womens': 0,
    'youth': 0
}

In [43]:
save_skiboard(skiboard)

In [44]:
print(skiboard['id'])

97


# Add Sizes to SkiBoard
---

In [45]:
table = input("Please submit your ski/snowboard size table\n\n").lower()

Please submit your ski/snowboard size table

Size (cm)	151	154	157	158W	160	161W	163 Effective Edge (mm)	1152	1177	1203	1203	1215	1215	1241 Tip Width (mm)	293	296	299	310	301	313	305 Waist Width (mm)	248	250	252	263	254	266	257 Tail Width (mm)	293	296	299	310	301	313	305 Sidecut Radius (m)	8.2 ​​/ 6.9 ​​/ 7.9	8.3 ​​/ 7 ​​/ 8	8.5 ​​/ 7.2 ​​/ 8.2	8.4 ​​/ 7.1 ​​/ 8.1	8.6 ​​/ 7.3 ​​/ 8.3	8.6 ​​/ 7.3 ​​/ 8.3	8.8 ​​/ 7.5 ​​/ 8.5 Stance Setback (mm)	19	19	19	19	19	19	19 Rider Weight (lbs)	100-150	125-175	130-180	150-220​​+	140-190	160-220​​+	160-210 Width	Regular	Regular	Regular	Wide	Regular	Wide	Regular


In [86]:
values = extract_data(table)

Row: size 	151	154	157	158w	160	161w	163 
Row: effective edge 	1152	1177	1203	1203	1215	1215	1241 
Row: tip width 	293	296	299	310	301	313	305 
Row: waist width 	248	250	252	263	254	266	257 
Row: tail width 	293	296	299	310	301	313	305 
Row: sidecut radius 	8.2 ​​/ 6.9 ​​/ 7.9	8.3 ​​/ 7 ​​/ 8	8.5 ​​/ 7.2 ​​/ 8.2	8.4 ​​/ 7.1 ​​/ 8.1	8.6 ​​/ 7.3 ​​/ 8.3	8.6 ​​/ 7.3 ​​/ 8.3	8.8 ​​/ 7.5 ​​/ 8.5 
 	8.2 ​​/ 6.9 ​​/ 7.9	8.3 ​​/ 7 ​​/ 8	8.5 ​​/ 7.2 ​​/ 8.2	8.4 ​​/ 7.1 ​​/ 8.1	8.6 ​​/ 7.3 ​​/ 8.3	8.6 ​​/ 7.3 ​​/ 8.3	8.8 ​​/ 7.5 ​​/ 8.5 


In [87]:
for row in values:
    print(f"{row}:\n{values[row]}\n\n")

size:
['151', '154', '157', '158w', '160', '161w', '163']


nose width:
['293', '296', '299', '310', '301', '313', '305']


waist width:
['248', '250', '252', '263', '254', '266', '257']


tail width:
['293', '296', '299', '310', '301', '313', '305']


sidecut:
['8.2', '/', '6.9', '/', '7.9', '8.3', '/', '7', '/', '8', '8.5', '/', '7.2', '/', '8.2', '8.4', '/', '7.1', '/', '8.1', '8.6', '/', '7.3', '/', '8.3', '8.6', '/', '7.3', '/', '8.3', '8.8', '/', '7.5', '/', '8.5']


setback:
['19', '19', '19', '19', '19', '19', '19']


effective edge:
['1152', '1177', '1203', '1203', '1215', '1215', '1241']




In [29]:
success = save_values(skiboard, values)
if success:
    print("\nSUCCESS!")
    id_counter += 1


SUCCESS!
